# **1. Mô tả ứng dụng** 

## **Ứng dụng mà nhóm chọn**

Huấn luyện Convolutional Neural Network để phân lớp ảnh bằng Stochastic Gradient Descent.
*   **Input:** Tập dữ liệu ảnh và nhãn của ảnh, cấu trúc của CNN và các tham số liên quan.

*   **Output:** Mô hình phân lớp ảnh sau khi huấn luyện.

*   **Ý nghĩa của ứng dụng trong thực tế:** Giúp số hoá tài liệu, OCR...

## **Ứng dụng này có cần phải tăng tốc không?**

Có, vì tập dữ liệu lớn và có nhiều hidden layer nên chạy rất chậm nếu chạy tuần tự.

## **Ứng dụng này có thể tăng tốc bằng cách song song hóa không?**

Có, vì trong đó có phép convolution, phép nhân ma trận, phép max-pooling,...

# **2. Cài đặt tuần tự**

## **Thiết kế**

*   Trộn ngẫu nhiên thứ tự ảnh.

*   Đối với mỗi bức ảnh đầu vào, ta sẽ chuyển nó về mảng numpy.

*   Nếu là ảnh RGB thì ta sẽ chuyển nó về ảnh grayscale.

*   Tập ảnh sẽ được chia thành nhiều batch có kích thước giống nhau và lớn hơn hoặc bằng 1, batch cuối cùng có thể có kích thước khác.

*   Với mỗi batch được đưa vào, ta sẽ chuẩn hoá các mảng numpy trong batch về khoảng [-0.5,0.5] để tránh việc giá trị các lớp sau trở nên quá lớn.

*   CNN sẽ có 1 tầng convolutional, 1 tầng max-pooling, 1 tầng fully connected và activation là hàm softmax.

    *   Tầng convolution dùng để nhận dạng các đặc trưng cụ thể hoặc hình mẫu trong bức ảnh.

    *   Tầng max-pooling dùng để tập hợp các đặc trưng nhỏ lẻ của ảnh thành một đặc trưng lớn hơn, ít nhạy cảm với vị trí của đặc trưng hơn.

    *   Tầng fully connected dùng để quyết định nhãn của ảnh.

    ![](https://victorzhou.com/media/cnn-post/cnn-dims-3.svg)

    <center><small>Nguồn: https://victorzhou.com/blog/intro-to-cnns-part-1/</small></center>

*   **Lan truyền xuôi:**

    *   ***Tầng convolution:***

        Trong tầng này, ta sẽ có $n$ node tương đương với $n$ filter với kích thước là $m$x$m$, trọng số của các filter này sẽ được khởi tạo ngẫu nhiên và mỗi trọng số đó sẽ được chia cho $m$x$m$ để chống việc kết quả ra số quá lớn.

        Ta sẽ lần lượt trượt các filter từ phải sang trái, từ trên xuống dưới, mỗi lần một pixel. Với mỗi pixel tương ứng trong cửa sổ trượt, ta sẽ nhân giá trị của nó với trọng số trong filter rồi cộng lại với nhau.

        ![](https://drive.google.com/uc?export=view&id=1riGOZlVlZdE-iAId6b23kaMLz7NncRT-)

        ![](https://drive.google.com/uc?export=view&id=1M7zZWBg4Hr_aJ7OJZ1t_Y4s48DuYUSkq)

    *   ***Tầng max-pooling:***

        Với pool có kích thước là $p$x$p$, ta cũng lần lượt trượt các filter từ phải sang trái, từ trên xuống dưới, nhưng mỗi lần là $p$ pixel. Ta sẽ chọn ra giá trị lớn nhất trong cửa sổ trượt đó.

        ![](https://drive.google.com/uc?export=view&id=1soCPEqlOsBjZNTap4opnfFF5rehv0anc)

    *   ***Tầng fully connected:***

        Ta sẽ có $k$ node tương đương với $k$ lớp cần phân, vì đây là tầng fully connected nên tất cả các node của tầng trước sẽ được nối với tất cả các node trong tầng này. Tức là giá trị đầu ra của mỗi node sẽ được tính bằng công thức sau: $t = w * x + b$ với $t$ là giá trị đầu ra của mỗi node, $w$ là vector chứa trọng số của node đó, $x$ là vector các giá trị đầu ra của tất cả các node ở tầng trước đó và $b$ là bias của node đó. Với $k$ node ta sẽ thu được một vector $k$ chiều và ta sẽ tính softmax của vector này, mỗi giá trị sẽ tương ứng với xác suất của mỗi lớp và ta sẽ chọn lớp nào có xác suất cao nhất.

        Ta sẽ dùng độ đo cross-entropy để tính độ lỗi: $L=-\text{ln}(p_c)$ với $p_c$ là xác suất của lớp đúng $c$.

*   **Lan truyền ngược:**

    Mục tiêu của việc học chỉ đơn giản là tối thiểu hoá độ lỗi. Để làm được điều đó, ta sẽ sử dụng kĩ thuật là Gradient Descent (GD), tức là ta phải tính gradient của hàm lỗi. Ở đây ta sẽ sử dụng một biến thể của GD là Stochastic Gradient Descent (SGD).
    SGD cũng giống như GD nhưng thay vì cập nhật các trọng số và bias sau khi duyệt hết tập train, ta sẽ cập nhật với mỗi một batch được duyệt.

    Trong quá trình này ta sẽ lần lượt tính các gradient của từng tầng bằng quy tắt mắc xích để cập nhật các trọng số và bias trong các tầng đó.

    *   ***Tầng fully connected:***

        Ta có thể dùng quy tắc mắc xích để tính các gradient sau của mỗi bức ảnh:
        \begin{equation}
          \frac{\partial L}{\partial w} = \frac{\partial L}{\partial {\text{output}_\text{FCL}}} * \frac{\partial {\text{output}_\text{FCL}}}{\partial t} * \frac{\partial t}{\partial w}
        \end{equation}
        \begin{equation}
          \frac{\partial L}{\partial b} = \frac{\partial L}{\partial {\text{output}_\text{FCL}}} * \frac{\partial {\text{output}_\text{FCL}}}{\partial t} * \frac{\partial t}{\partial b}
        \end{equation}
        \begin{equation}
          \frac{\partial L}{\partial {\text{output}_\text{MPL}}} = \frac{\partial L}{\partial {\text{output}_\text{FCL}}} * \frac{\partial {\text{output}_\text{FCL}}}{\partial t} * \frac{\partial t}{\partial {\text{output}_\text{MPL}}}
        \end{equation}

        Để tính gradient cho toàn bộ 1 batch, ta tính trung bình cộng các gradient của tất cả các ảnh có trong batch đó.

        Sau đó, ta sẽ cập nhật các trọng số sau mỗi một batch được duyệt: $w_i = w_i - \alpha * \frac{\partial L}{\partial w}$ với $w_i$ là trọng số thứ $i$ và $\alpha$ là hệ số học.

        Tương tự, ta cũng cập nhật các bias: $b_i = b_i - \alpha * \frac{\partial L}{\partial b}$ với $b_i$ là bias thứ $i$ và $\alpha$ là hệ số học.

        Cuối cùng ta truyền $\frac{\partial L}{\partial {\text{output}_\text{MPL}}}$ về cho tầng max-pooling.

        Chi tiết các công thức có trong tài liệu tham khảo [2].

    *   ***Tầng max-pooling:***

        Vì tầng này không có trọng số nên ta không cần cập nhật gì cả. Ta chỉ cần khôi phục lại kích thước ban đầu của mảng trước khi đi qua tầng max-pooling, rồi gán $\frac{\partial L}{\partial {\text{output}_\text{MPL}}}$ vào vị trí có giá trị lớn nhất trong cửa sổ trượt $p$x$p$ ban đầu, những vị trí còn lại sẽ được gán bằng 0. Đây chính là $\frac{\partial L}{\partial {\text{output}_\text{CL}}}$.

        ![](https://victorzhou.com/media/cnn-post/maxpool-backprop.svg)

        <center><small>Nguồn: https://victorzhou.com/blog/intro-to-cnns-part-2/</small></center>

        Ta sẽ truyền $\frac{\partial L}{\partial {\text{output}_\text{CL}}}$ về cho tầng convolution.

        Việc này được thực hiện độc lập nhau với tất cả các ảnh có trong 1 batch.

    *   ***Tầng convolutional:***

        Ta cũng dùng quy tắc mắc xích để tính gradient sau:
        \begin{equation}
          \frac{\partial L}{\partial \text{filter}(x,y)} = \sum\limits_{i}\sum\limits_{j}\frac{\partial L}{\partial {\text{output}_\text{CL}(i,j)}} * \frac{\partial {\text{output}_\text{CL}(i,j)}}{\partial \text{filter}(x,y)}
        \end{equation}

        Tương tự, để tính gradient cho toàn bộ 1 batch, ta tính trung bình cộng các gradient của tất cả các ảnh có trong batch đó.

        Ta cập nhật các trọng số của filter: $\text{filter}(x,y) = \text{filter}(x,y) - \alpha * \frac{\partial L}{\partial \text{filter}(x,y)}$ với $\text{filter}(x,y)$ là trọng số của filter ở dòng $x$ cột $y$ và $\alpha$ là hệ số học.

        Chi tiết các công thức có trong tài liệu tham khảo [2].

*   Lặp lại cho tất cả các ảnh còn lại.
    
*   Nếu đã duyệt hết ảnh, ta sẽ thực hiện lại từ đầu, 1 vòng như vậy được gọi là 1 epoch.

*   Sau khi thực hiện tất cả các epoch, ta sẽ thu được một mô hình với các trọng số đã được cập nhật.

## **Đánh giá**

### **Cấu hình**

*   Thử nghiệm trên tập dữ liệu chữ số viết tay MNIST. Tập dữ liệu này chứa các ảnh chữ số viết tay từ 0 đến 9, nó bao gồm 60000 ảnh trong tập train và 10000 ảnh trong tập test, mỗi ảnh có kích thước là 28x28 và là ảnh grayscale.

*   Cấu hình của CNN như sau:

  *   Tầng convolutional sẽ có 32 filter, mỗi filter sẽ có kích thước là 5x5.

  *   Tầng max pool có kích thước của pool là 2x2.

  *   Tầng fully connected sẽ có 10 node tương ứng với 10 chữ số từ 0 đến 9.

  *   $\alpha = 0.005$

  *   epoch = 10

  *   batch size = 100

### **Nhận xét**

  *   Thời gian huấn luyện khoảng 16 phút 22 giây cho 10 epoch với mỗi epoch khoảng 98 giây. Tỷ lệ chính xác khi train đạt khoảng 90% và tỷ lệ chính xác khi test đạt khoảng 91%.

  *   Thời gian huấn luyện của keras nhanh hơn, chỉ 3 phút 40 giây cho 10 epoch với mỗi epoch khoảng 22 giây. Tỷ lệ chính xác khi train đạt khoảng 92% và tỷ lệ chính xác khi test cũng đạt khoảng 92%.

  *   Nhóm vẫn chưa biết tại sao lại có sự khác biệt này.

### **Kết quả**

In [ ]:
!python source.py

2021-06-23 06:11:05.257866: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Loading data...
Normalizing...
Initiating parameters...
Compiling...

--------------Our model--------------

Epoch 1/10
	98s - loss: 0.4297 - accuracy: 0.8779 - val_loss: 0.3730 - val_accuracy: 0.8916
Epoch 2/10
	98s - loss: 0.3769 - accuracy: 0.8897 - val_loss: 0.3419 - val_accuracy: 0.9007
Epoch 3/10
	98s - loss: 0.3557 - accuracy: 0.8957 - val_loss: 0.3245 - val_accuracy: 0.9068
Epoch 4/10
	98s - loss: 0.3438 - accuracy: 0.8987 - val_loss: 0.3183 - val_accuracy: 0.9086
Epoch 5/10
	98s - loss: 0.3357 - accuracy: 0.9017 - val_loss: 0.3143 - val_accuracy: 0.9099
Epoch 6/10
	97s - loss: 0.3293 - accuracy: 0.9023 - val_loss: 0.3077 - val_accuracy: 0.9121
Epoch 7/10
	97s - loss: 0.3241 - accuracy: 0.9051 - val_loss: 0.3056 - val_accuracy: 0.9127
Epoch 8/10
	98s - loss: 0.3198 - accuracy: 0.9061 - val_loss: 0.2993 - val_accuracy: 0.9138
Epoch 9/1

# **3. Cài đặt song song (trên GPU)**

## **Phân tích**


Các phép convolution, max-pooling và nhân ma trận chiếm nhiều thời gian thực thi nhất và các phép toán trong đó hoàn toàn độc lập với nhau nên ta có thể song song hoá 3 thao tác này.

##**Thiết kế**

Tránh việc phải di chuyển dữ liệu qua lại giữa CPU và GPU nhiều nhất có thể.

Đối với phép nhân ma trận, ta sẽ sử dụng block 2 chiều và grid 2 chiều, với mỗi thread sẽ tính một phần tử trong ma trận output.

Đối với phép convolution và max-pooling, ta sẽ sử dụng block 2 chiều và grid 3 chiều, với mỗi thread sẽ tính một phần tử trong ma trận output và chiều z của grid sẽ ứng với mỗi neuron.

Mỗi hàm kernel sẽ chỉ xử lý đúng 1 ảnh được đưa vào.

## **Đánh giá**

### **Cấu hình**

*   Thử nghiệm trên tập dữ liệu chữ số viết tay MNIST. Tập dữ liệu này chứa các ảnh chữ số viết tay từ 0 đến 9, nó bao gồm 60000 ảnh trong tập train và 10000 ảnh trong tập test, mỗi ảnh có kích thước là 28x28 và là ảnh grayscale.

*   Cấu hình của CNN như sau:

  *   Tầng convolutional sẽ có 32 filter, mỗi filter sẽ có kích thước là 5x5.

  *   Tầng max pool có kích thước của pool là 2x2.

  *   Tầng fully connected sẽ có 10 node tương ứng với 10 chữ số từ 0 đến 9.

  *   $\alpha = 0.005$

  *   epoch = 10

  *   batch size = 100

  *   **block size = (32, 32)**

### **Nhận xét**

Mặc dù chương trình chạy đúng nhưng thời gian thực thi không được rút ngắn như mong đợi, thậm chí còn chậm hơn rất nhiều lần so với host, với tổng thời gian thực thi là 1 tiếng 3 phút 40 giây, mỗi epoch chiếm khoảng 382 giây.

Việc này là do vòng lặp của python quá chậm, ta cũng không thể jit được hàm gọi hàm kernel nên ta sẽ cần một thiết kế khác hiệu quả hơn.

### **Kết quả**

In [ ]:
!python source_cuda_ver_1.py

2021-07-03 05:02:19.841767: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Loading data...
Shuffle training data
Normalizing and copying data to GPU...
Initiating parameters...
Start training...

--------------Our model--------------

Epoch 1/10
	385s - loss: 0.9879 - accuracy: 0.7543 - val_loss: 0.4640 - val_accuracy: 0.8780
Epoch 2/10
	382s - loss: 0.4274 - accuracy: 0.8784 - val_loss: 0.3688 - val_accuracy: 0.8954
Epoch 3/10
	382s - loss: 0.3754 - accuracy: 0.8899 - val_loss: 0.3399 - val_accuracy: 0.9036
Epoch 4/10
	381s - loss: 0.3546 - accuracy: 0.8960 - val_loss: 0.3254 - val_accuracy: 0.9066
Epoch 5/10
	382s - loss: 0.3425 - accuracy: 0.8993 - val_loss: 0.3165 - val_accuracy: 0.9095
Epoch 6/10
	381s - loss: 0.3342 - accuracy: 0.9013 - val_loss: 0.3102 - val_accuracy: 0.9105
Epoch 7/10
	381s - loss: 0.3279 - accuracy: 0.9040 - val_loss: 0.3054 - val_accuracy: 0.9113
Epoch 8/10
	382s - loss: 0.3228 - accuracy:

# **4. Cài đặt song song (trên GPU) + tối ưu hóa**

## **4.1. Tối ưu hoá lần 1**

### **Phân tích**

Vì vòng lặp trong python rất chậm nên ta sẽ đưa vòng lặp đó vào trong hàm kernel, nghĩa là mỗi hàm kernel sẽ xử lý cả 1 batch.

### **Thiết kế**

Ta chỉ cần thêm 1 vòng lặp vào các hàm kernel như phiên bản đầu để một hàm kernel có thể xử lý nhiều ảnh trong 1 lần gọi và tránh xử dụng vòng lặp của python nhiều nhất có thể.

Riêng việc cập nhật các trọng số, ta sẽ sao chép dữ liệu ra CPU rồi mới tiến hành cập nhật, lý do sẽ được nói rõ ở phần sau.

### **Đánh giá**

#### **Cấu hình**

Các thông số cấu hình đều được giữ nguyên.

#### **Nhận xét**

Chỉ cần 1 thay đổi rất đơn giản này mà thời gian thực thi đã giảm đi rất rất nhiều, tổng thời gian thực thi chỉ còn 2 phút 6 giây với mỗi epoch chiếm khoảng 12 giây.

#### **Kết quả**

In [ ]:
!python source_cuda_ver_2.py

2021-07-03 06:06:03.959580: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Loading data...
Shuffle training data
Normalizing and copying data to GPU...
Initiating parameters...
Start training...

--------------Our model--------------

Epoch 1/10
	16s - loss: 0.9889 - accuracy: 0.7607 - val_loss: 0.4637 - val_accuracy: 0.8773
Epoch 2/10
	12s - loss: 0.4275 - accuracy: 0.8800 - val_loss: 0.3686 - val_accuracy: 0.8942
Epoch 3/10
	12s - loss: 0.3749 - accuracy: 0.8907 - val_loss: 0.3396 - val_accuracy: 0.9026
Epoch 4/10
	12s - loss: 0.3540 - accuracy: 0.8958 - val_loss: 0.3250 - val_accuracy: 0.9071
Epoch 5/10
	12s - loss: 0.3418 - accuracy: 0.8991 - val_loss: 0.3159 - val_accuracy: 0.9097
Epoch 6/10
	12s - loss: 0.3334 - accuracy: 0.9019 - val_loss: 0.3094 - val_accuracy: 0.9110
Epoch 7/10
	12s - loss: 0.3270 - accuracy: 0.9039 - val_loss: 0.3044 - val_accuracy: 0.9123
Epoch 8/10
	12s - loss: 0.3218 - accuracy: 0.9057 

### **Lý do cho việc không thực hiện cập nhật trong device**

Do sự khác biệt về cách tính toán số floating-point nên kết quả khi chạy trên host sẽ khác với kết quả khi chạy trên device. Mặc dù sự khác biệt là rất nhỏ nhưng khi đi qua nhiều lặp thì sự khác biệt này sẽ trở nên rõ rệt và sẽ ảnh hưởng rất lớn đến độ chính xác của chương trình (từ 87% xuống 16%). Ngoài ra ở phiên bản này vì gặp phải hiện tượng tràn số nên phải giảm learningRate xuống = 0.0001.

Mặc dù ở phiên bản dưới đây có thời gian thực thi nhanh hơn phiên bản trên 2 giây nhưng việc này là không đáng kể và tiềm ẩn nhiều lỗi không thể ngờ tới.

In [ ]:
!python source_cuda_ver_3.py

2021-07-03 06:08:13.623291: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Loading data...
Shuffle training data
Normalizing and copying data to GPU...
Initiating parameters...
Start training...

--------------Our model--------------

Epoch 1/10
	14s - loss: 2.3025 - accuracy: 0.1058 - val_loss: 2.2297 - val_accuracy: 0.1637
Epoch 2/10
	10s - loss: 2.2309 - accuracy: 0.1596 - val_loss: 2.1658 - val_accuracy: 0.2849
Epoch 3/10
	10s - loss: 2.1675 - accuracy: 0.2743 - val_loss: 2.1003 - val_accuracy: 0.4185
Epoch 4/10
	10s - loss: 2.1017 - accuracy: 0.4016 - val_loss: 2.0306 - val_accuracy: 0.5141
Epoch 5/10
	10s - loss: 2.0315 - accuracy: 0.4922 - val_loss: 1.9555 - val_accuracy: 0.5795
Epoch 6/10
	10s - loss: 1.9556 - accuracy: 0.5560 - val_loss: 1.8744 - val_accuracy: 0.6236
Epoch 7/10
	10s - loss: 1.8738 - accuracy: 0.6038 - val_loss: 1.7876 - val_accuracy: 0.6580
Epoch 8/10
	10s - loss: 1.7864 - accuracy: 0.6391 

## **4.2. Tối ưu hoá lần 2**

### **Phân tích**

Xét thấy trong hàm convolution layer kernel, mỗi thread đều truy cập cùng một vùng nhớ nhiều lần ở GMEM nên ta có thể tải dữ liệu lên SMEM để giảm thiểu số lần truy cập vào GMEM, từ đó tăng tốc độ tính toán.

### **Thiết kế**

Ta thêm biến là mảng dữ liệu gọi trong hàm SMEM sau đó chép dữ liệu từ bên GMEM qua SMEM rồi khi truy xuất dữ liệu cần từ mảng SMEM này trong 2 hàm convolution layer

### **Đánh giá**

### **Kết quả**

In [ ]:
!python source_cuda_ver_4.py

2021-07-09 11:50:11.225538: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Loading data...
11493376/11490434 [==============================] - 0s 0us/step
Shuffle training data
Normalizing and copying data to GPU...
Initiating parameters...
Start training...

--------------Our model--------------

Epoch 1/10
	16s - loss: 1.4629 - accuracy: 0.6874 - val_loss: 1.0031 - val_accuracy: 0.8088
Epoch 2/10
	12s - loss: 0.8625 - accuracy: 0.8121 - val_loss: 0.7129 - val_accuracy: 0.8416
Epoch 3/10
	12s - loss: 0.6715 - accuracy: 0.8386 - val_loss: 0.5911 - val_accuracy: 0.8559
Epoch 4/10
	12s - loss: 0.5810 - accuracy: 0.8528 - val_loss: 0.5266 - val_accuracy: 0.8657
Epoch 5/10
	12s - loss: 0.5295 - accuracy: 0.8615 - val_loss: 0.4874 - val_accuracy: 0.8726
Epoch 6/10
	12s - loss: 0.4970 - accuracy: 0.8678 - val_loss: 0.4617 - val_accuracy: 0.8770
Epoch 7/10
	12s - loss: 0.4749 - accuracy: 0.8722 - val_loss: 0.4435 - val_ac

# **5. Nhìn lại quá trình làm đồ án**

## **Đã gặp những khó khăn gì**

*   Lương Bội Sương: phải làm quen với các cú pháp của ngôn ngữ mới và thường gặp những lỗi không rõ ràng hoặc đôi khi còn gây nhầm lẫn nên thường đi vào bế tắc.
*   Phạm Như Quyền: lúc đầu còn bỡ ngỡ khi làm quen với thư viện numba và tìm hiểu về các cấu trúc mạng trong đồ án.
*   Trần Đình Sang: Giai đoạn đầu do chưa có nền tảng về thư viện và ngôn ngữ nên chậm hơn so với nhịp độ chung của cả nhóm cũng như là trong lớp. 
*   Võ Thế Sơn: việc làm quen với python giai đoạn đầu mất khá nhiều thời gian, các lỗi trong quá trình code mất nhiều thời gian để sửa.

## **Học được những gì hữu ích**

*   Lương Bội Sương: không nên hấp tấp, phải đi từ đơn giản đến phức tạp. Phải có những khoảng nghỉ ngắn để đầu óc được thư giản mới có thể nghĩ ra được ý tưởng mới. Cuối cùng là nhờ đến sự giúp đỡ của người khác khi cần.
*   Phạm Như Quyền: đã hiểu được cách sử dụng thư viện và hiểu thêm được quá trình xử lý song song trong một đồ án lớn. Luôn cố gắng phối hợp với các thành viên trong nhóm khi cần.
*   Trần Đình Sang: cách chia công việc ra thành nhiều phần và nhiều giai đoạn để dễ thực hiện và tận dụng được lợi thế làm việc nhóm, bên cạnh đó cần có những bước nhìn lại cũng như là thay đổi hướng đi khi cần. 
*   Võ Thế Sơn: biết được cách song song hoá cơ bản sử dụng ngôn ngữ python, cũng cố thêm kiến thức về lập trình song song trên GPU.

## **Nếu có thên thời gian thì nhóm sẽ làm gì**

Nhóm sẽ nghiên cứu thêm về mã nguồn của keras để có kết quả giống nhau hơn, tìm hiểu nhiều hơn về kĩ thuật trong học sâu và cấu trúc của CNN để đạt được độ chính xác cao hơn.

Tìm cách đưa vào STREAM và hoàn thiện phiên bản sử dụng SMEM.

# **6. Tài liệu tham khảo**

[1] https://victorzhou.com/blog/intro-to-cnns-part-1/

[2] https://victorzhou.com/blog/intro-to-cnns-part-2/

[3] https://www.kaggle.com/moltean/fruits